In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from vkoga_2L import VKOGA_2L
import tkernels
import kernels

In [2]:
def generate_data(n_samples=500, n_features=2, n_targets=1):
    desired_A = torch.tensor([[0.5, 0.0], [0.0, 2.0]])  # True transformation matrix to calc X and compare later to learned A
    X = torch.randn(n_samples, n_features)
    X_transformed = X @ desired_A
    y = X_transformed[:, 0] + X_transformed[:, 1] # + torch.randn(1000)
    return X, y, desired_A

In [3]:
# Generate data
X, Y, desired_A = generate_data(n_samples=500, n_features=2, n_targets=1)

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
# Initialize VKOGA_2L model (some Parameters are described in Page 123)
model = VKOGA_2L(
    kernel=[kernels.Gaussian(), tkernels.Gaussian()], # quadratic Matern kernel used
    flag_2L_optimization=True,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-5,
    learning_rate=1e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

In [5]:
# Fit the model on training data
model.fit(X_train, y_train)

torch.Size([400, 2])
torch.Size([400])


VKOGA_2L(flag_2L_optimization=True, flag_optim_verbose=False,
         kernel=<kernels.Gaussian object at 0x000002C5E8534DC0>,
         learning_rate=0.001, n_epochs_optim=1000, reg_para_optim=1e-05,
         restr_par=0.01, verbose=False)

In [7]:
# Print some predictions and ground truth
predictions = model.predict(X_test)
predictions = predictions.flatten()
print("Predictions:", predictions[:5])
print("Ground truth:", y_test[:5])

# Print Error
mse = np.mean((predictions - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")

Predictions: [ 3.35413676  2.01652845  3.57601069  0.88041726 -1.44048157]
Ground truth: tensor([ 3.3541,  2.0165,  3.5760,  0.8804, -1.4405])
MSE on Test Data: 0.000000


In [8]:
# No use of 2 Layer Vkoga (No learned A => A is identity)
# Initialize VKOGA_2L model (some Parameters are described in Page 123)
model = VKOGA_2L(
    kernel=[kernels.Gaussian(), tkernels.Gaussian()], # quadratic Matern kernel used
    flag_2L_optimization=False,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-5,
    learning_rate=1e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

If kernel optimization is desired, set flag_2L_optimization to True!


In [9]:
# Fit the model on training data
model.fit(X_train, y_train)

torch.Size([400, 2])
torch.Size([400])


VKOGA_2L(flag_optim_verbose=False,
         kernel=<kernels.Gaussian object at 0x000002C5E9CE4D30>,
         learning_rate=0.001, n_epochs_optim=1000, reg_para_optim=1e-05,
         restr_par=0.01, verbose=False)

In [10]:
# Print some predictions and ground truth
predictions = model.predict(X_test)
predictions = predictions.flatten()
print("Predictions:", predictions[:5])
print("Ground truth:", y_test[:5])

# Print Error
mse = np.mean((predictions - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")

Predictions: [ 3.35400949  2.01251993  3.58001578  0.87993271 -1.44031479]
Ground truth: tensor([ 3.3541,  2.0165,  3.5760,  0.8804, -1.4405])
MSE on Test Data: 0.028439
